In [1]:
# VERİ HAZIRLIĞI
# VERİNİN S3'E TAŞINMASI
# MODEL OLUŞUMU
# MODEL EĞİTİMİ
# MODELİN DAĞITIMI

# VERİ HAZIRLIĞI

In [2]:
import pandas as pd
import urllib.request
urllib.request.urlretrieve('https://archive.ics.uci.edu/static/public//53/iris.zip', 'data.zip')

('data.zip', <http.client.HTTPMessage at 0x7f769d67f880>)

In [3]:
!mkdir data
!unzip data.zip -d data/

Archive:  data.zip
  inflating: data/Index              
  inflating: data/bezdekIris.data    
  inflating: data/iris.data          
  inflating: data/iris.names         


In [57]:
# veriyi okumak
df = pd.read_csv('data/iris.data', header=None)
# data = pd.read_csv('data/iris.data', header = None)

# sayısal değerlere dönüştürmek
data[4] = data[4].replace('Iris-setosa', 0)
data[4] = data[4].replace('Iris-virginica', 1)
data[4] = data[4].replace('Iris-versicolor', 2)

# karıştırmak
data = data.sample(frac = 1).reset_index(drop = True)

# bölmek (eğitim ve doğrulama veri kümeleri)
# %80 eğitim veri seti / %20 doğrulama veri seti
train_data = data[:120]
val_data = data[120:]

# VERİNİN S3'E TAŞINMASI

In [58]:
import boto3
bucket_name = 'sagemaker-kurma-ve-dagitma-modeli'

data = pd.concat([data[4], data.drop(columns=[4])], axis=1)
train_data.to_csv('data.csv', header = False, index = False)
key = 'data/train/data'
url = 's3://{}/{}'.format(bucket_name, key)
boto3.Session().resource('s3').Bucket(bucket_name).Object(key).upload_file('data.csv')

val_data.to_csv('data.csv', header = False, index = False)
key = 'data/val/data'
url = 's3://{}/{}'.format(bucket_name, key)
boto3.Session().resource('s3').Bucket(bucket_name).Object(key).upload_file('data.csv')

[05/11/25 16:38:10] INFO     Found credentials from IAM Role:                                   ]8;id=475229;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=850043;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

                    INFO     Found credentials from IAM Role:                                   ]8;id=235279;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=832479;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

In [59]:
import boto3

bucket_name = 'sagemaker-kurma-ve-dagitma-modeli'

data = pd.concat([data[4], data.drop(columns=[4])], axis=1)

train_data = data[:120]
val_data = data[120:]

train_data.to_csv('train_data.csv', header=False, index=False)
train_key = 'data/train/data.csv'
train_url = f's3://{bucket_name}/{train_key}'
boto3.Session().resource('s3').Bucket(bucket_name).Object(train_key).upload_file('train_data.csv')

val_data.to_csv('val_data.csv', header=False, index=False)
val_key = 'data/val/data.csv'
val_url = f's3://{bucket_name}/{val_key}'
boto3.Session().resource('s3').Bucket(bucket_name).Object(val_key).upload_file('val_data.csv')


[05/11/25 16:38:16] INFO     Found credentials from IAM Role:                                   ]8;id=210122;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=683707;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

                    INFO     Found credentials from IAM Role:                                   ]8;id=251892;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=102792;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

# MODEL OLUŞUMU

In [48]:
import sagemaker
from sagemaker.amazon.amazon_estimator import get_image_uri
from sagemaker import get_execution_role
from sagemaker import image_uris

key = 'model/xgb_model'
s3_output_location = 's3://{}/{}'.format(bucket_name, key)

container = image_uris.retrieve(framework='xgboost', region=boto3.Session().region_name, version='1.7-1')
xgb_model = sagemaker.estimator.Estimator(
#    image_uri=container,
    get_image_uri(boto3.Session().region_name, 'xgboost'),
    role = get_execution_role(),
    train_instance_count = 1,
    train_instance_type = 'ml.m5.xlarge',
    train_volume_size = 5,
    output_path = s3_output_location,
    sagemaker_session = sagemaker.Session())

xgb_model.set_hyperparameters(max_depth = 5,
    eta = 0.2,
    gamma = 4,
    min_child_weight = 6,
#    silent = 0,
    verbosity = 1,
    objective = 'multi:softmax',
    num_class = 3,
    num_round = 10)

[05/11/25 16:18:07] INFO     Ignoring unnecessary instance type: None.                            ]8;id=933051;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/image_uris.py\image_uris.py]8;;\:]8;id=863317;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/image_uris.py#530\530]8;;\

                    WARNING  The method get_image_uri has been renamed in sagemaker>=2.          ]8;id=255842;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/deprecations.py\deprecations.py]8;;\:]8;id=668573;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/deprecations.py#34\34]8;;\
                             See: https://sagemaker.readthedocs.io/en/stable/v2.html for                           
                             details.                                                                              

                    INFO     Ignoring unnecessary instance type: None.                            ]8;id=729171;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/image_uris.py\image_uris.py]8;;\:]8;id=377847;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/image_uris.py#530\530]8;;\

                    WARNING  train_instance_count has been renamed in sagemaker>=2.              ]8;id=994594;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/deprecations.py\deprecations.py]8;;\:]8;id=552352;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/deprecations.py#34\34]8;;\
                             See: https://sagemaker.readthedocs.io/en/stable/v2.html for                           
                             details.                                                                              

                    WARNING  train_instance_type has been renamed in sagemaker>=2.               ]8;id=205379;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/deprecations.py\deprecations.py]8;;\:]8;id=238872;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/deprecations.py#34\34]8;;\
                             See: https://sagemaker.readthedocs.io/en/stable/v2.html for                           
                             details.                                                                              

                    WARNING  train_volume_size has been renamed in sagemaker>=2.                 ]8;id=504265;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/deprecations.py\deprecations.py]8;;\:]8;id=569422;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/deprecations.py#34\34]8;;\
                             See: https://sagemaker.readthedocs.io/en/stable/v2.html for                           
                             details.                                                                              

# MODEL EĞİTİMİ

In [49]:
import numpy as np
labels = data[4]
unique_labels = sorted(np.unique(labels))
label_mapping = {label: idx for idx, label in enumerate(unique_labels)}
data[4] = np.vectorize(label_mapping.get)(labels)
np.sort(np.unique(data[4]))

array([0, 1, 2])

In [53]:
import pandas as pd

df_train = pd.read_csv('data.csv', header=None)
print("Unique labels in training set:", df_train[0].unique())

df_val = pd.read_csv('data.csv', header=None)
print("Unique labels in validation set:", df_val[0].unique())

Unique labels in training set: [6.4 6.3 5.4 6.7 5.5 7.7 5.2 5.8 4.8 4.9 4.6 6.2 5.1 5.7 6.1 6.9 6.  5.6
 5.  6.6 4.4]
Unique labels in validation set: [6.4 6.3 5.4 6.7 5.5 7.7 5.2 5.8 4.8 4.9 4.6 6.2 5.1 5.7 6.1 6.9 6.  5.6
 5.  6.6 4.4]


In [60]:
from sagemaker.inputs import TrainingInput

train_data = 's3://{}/{}'.format(bucket_name, 'data/train')
val_data = 's3://{}/{}'.format(bucket_name, 'data/val')

# train_channel = sagemaker.session.s3_input(train_data, content_type = 'text/csv')
train_channel = TrainingInput(train_data, content_type='text/csv')
# val_channel = sagemaker.session.s3_input(val_data, content_type = 'text/csv')
val_channel = TrainingInput(val_data, content_type='text/csv')

data_channels = {'train' : train_channel, 'validation' : val_channel}
xgb_model.fit(inputs = data_channels)

[05/11/25 16:38:42] INFO     SageMaker Python SDK will collect telemetry to help us better  ]8;id=732439;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/telemetry/telemetry_logging.py\telemetry_logging.py]8;;\:]8;id=908926;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/telemetry/telemetry_logging.py#91\91]8;;\
                             understand our user's needs, diagnose issues, and deliver                             
                             additional features.                                                                  
                             To opt out of telemetry, please disable via TelemetryOptOut                           
                             parameter in SDK defaults config. For more information, refer                         
                             to                                                                                    
                             https://sagemaker.readthedocs.io/en/stable/overview.html#confi                        
                             guring-and-using-defaults-with-the-sagemaker-python-sdk.                              

                    INFO     Creating training-job with name: xgboost-2025-05-11-16-38-42-461       ]8;id=580717;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=155027;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py#1042\1042]8;;\

2025-05-11 16:38:42 Starting - Starting the training job...
2025-05-11 16:39:14 Starting - Preparing the instances for training...
2025-05-11 16:39:30 Downloading - Downloading input data...
2025-05-11 16:39:51 Downloading - Downloading the training image...
2025-05-11 16:40:47 Training - Training image download completed. Training in progress.
2025-05-11 16:40:47 Uploading - Uploading generated training modelArguments: train
[2025-05-11:16:40:41:INFO] Running standalone xgboost training.
[2025-05-11:16:40:41:INFO] File size need to be processed in the node: 0.01mb. Available memory size in the node: 8245.71mb
[2025-05-11:16:40:41:INFO] Determined delimiter of CSV input is ','
[16:40:41] S3DistributionType set as FullyReplicated
[16:40:41] 240x4 matrix with 960 entries loaded from /opt/ml/input/data/train?format=csv&label_column=0&delimiter=,
[2025-05-11:16:40:41:INFO] Determined delimiter of CSV input is ','
[16:40:41] S3DistributionType set as FullyReplicated
[16:40:41] 60x4 matrix w

In [51]:
df = pd.read_csv('data.csv')
print(df.head())
print(df.iloc[:, 0].unique())

   6.4  3.2  5.3  2.3  1
0  6.3  3.3  4.7  1.6  2
1  5.4  3.0  4.5  1.5  2
2  6.7  3.3  5.7  2.1  1
3  5.5  2.6  4.4  1.2  2
4  7.7  2.8  6.7  2.0  1
[6.3 5.4 6.7 5.5 7.7 6.4 5.2 5.8 4.8 4.9 4.6 6.2 5.1 5.7 6.1 6.9 6.  5.6
 5.  6.6 4.4]


# MODELİN DAĞITIMI

In [61]:
xgb_predictor = xgb_model.deploy(initial_instance_count = 1,
                                instance_type = 'ml.m5.xlarge')

[05/11/25 16:45:47] INFO     Creating model with name: xgboost-2025-05-11-16-45-47-848              ]8;id=244044;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=758303;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py#4094\4094]8;;\

[05/11/25 16:45:48] INFO     Creating endpoint-config with name xgboost-2025-05-11-16-45-47-848     ]8;id=848953;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=748631;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py#6019\6019]8;;\

                    INFO     Creating endpoint with name xgboost-2025-05-11-16-45-47-848            ]8;id=168667;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=396546;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py#4841\4841]8;;\

--------!